# Assignment2

## Notebook made by  

|** Name** | **Student id** | **email**|
|:- |:-|:-|
|Niek de Visscher | 10667474 | niekdevisscher@hetnet.nl |
| Leon Smits | 10500731 | leonmarijnsmits@gmail.com|
| Daniël van lier | 10639861 | danielvanlier@gmail.com |

### Pledge (taken from [Coursera's Honor Code](https://www.coursera.org/about/terms/honorcode) )



Put here a selfie with your photo where you hold a signed paper with the following text: (if this is team work, put two selfies here). The link must be to some place on the web, not to a local file. 

> My answers to homework, quizzes and exams will be my own work (except for assignments that explicitly permit collaboration).

>I will not make solutions to homework, quizzes or exams available to anyone else. This includes both solutions written by me, as well as any official solutions provided by the course staff.

>I will not engage in any other activities that will dishonestly improve my results or dishonestly improve/hurt the results of others.

<img src='http://i.imgur.com/hConduu.jpg'/>
<img src='http://i.imgur.com/r4tIPrA.jpg?1'/>
<img src='http://i.imgur.com/KbRwRFz.jpg'/>

### Note
* **Assignments without the selfies or completely filled in information will not be graded and receive 0 points.**


# Part 1: obtaining information from the web

### RSS parsing

Make a notebook that performs the following steps.

1. Create a script that retrieves all urls of rss feeds from <http://www.volkskrant.nl/rss/feeds/>. Use urllib2 and beautifulsoup for this. Store the urls in a list.
    * **update 2016**
    * As all Dutch sites, Volkskrant asks whether you accept cookies. This makes simple collecting webpages a lot harder. 
    * The code in the code cell below does the trick. 
    * After running this, I could collect further files from Volkskrant without additional cookie hassle.
2. Download all rss feeds and store them on disk.
3. Parse all RSS feeds using `lxml`. Create a list of  dicts with fields "channel", "url", "title", "date" in which you store this information for each item.
4. Compute some statistics about this dict: how many items, how many per channel, are there doubles (items occuring in several channels), etc.
5. Write this list as a csv file, store on disk, and upload to Google fusion tables.
6. Download all articles (once), parse out the text and store as pairs (date,text) in a list.
7. Count per day the number of words, and the number of unique words. Show this in a plot.

In [3]:
import cookielib # Thanks to http://stackoverflow.com/questions/29395407/enabling-cookies-with-urllib
import urllib2
import urllib
import requests
import os
from bs4 import BeautifulSoup
import re 


url = 'http://www.volkskrant.nl/rss/feeds/'

# with urllib2 and handling cookies
cookiejar= cookielib.LWPCookieJar()
opener= urllib2.build_opener( urllib2.HTTPCookieProcessor(cookiejar) )
response=opener.open(url)
html_doc= ' '.join(response.readlines())
 
rsssoup = BeautifulSoup(html_doc)


In [4]:
#Rss parsing:
#Question 1:
#rss list
rss_list = []
list_items=rsssoup.findAll('li')
len(list_items)
count = 0
for x in list_items:
    y = str(list_items[count])
    if "rss.xml" in y:
        url = re.search('"(.+?)"', y).group(1)
        rss_list.append(url)
    count += 1

print rss_list

['http://www.volkskrant.nl/nieuws/rss.xml', 'http://www.volkskrant.nl/nieuws-voorpagina/rss.xml', 'http://www.volkskrant.nl/buitenland/rss.xml', 'http://www.volkskrant.nl/binnenland/rss.xml', 'http://www.volkskrant.nl/opinie/rss.xml', 'http://www.volkskrant.nl/economie/rss.xml', 'http://www.volkskrant.nl/sport/rss.xml', 'http://www.volkskrant.nl/tech/rss.xml', 'http://www.volkskrant.nl/media/rss.xml', 'http://www.volkskrant.nl/wetenschap/rss.xml', 'http://www.volkskrant.nl/politiek/rss.xml', 'http://www.volkskrant.nl/cartoons/rss.xml', 'http://www.volkskrant.nl/foto/rss.xml', 'http://www.volkskrant.nl/vonk/rss.xml', 'http://www.volkskrant.nl/authors/rss.xml', 'http://www.volkskrant.nl/toplists/rss.xml', 'http://www.volkskrant.nl/archief/rss.xml', 'http://www.volkskrant.nl/cultuur-en-leven/rss.xml', 'http://www.volkskrant.nl/cultuur-en-leven-voorpagina/rss.xml', 'http://www.volkskrant.nl/recensies/rss.xml', 'http://www.volkskrant.nl/film/rss.xml', 'http://www.volkskrant.nl/muziek/rss.xm

In [5]:
#Question 2:
for url in rss_list:
    response = opener.open(url)
    html = response.read()
    soup = BeautifulSoup(html)
    title = soup.title.string[3:]
    with open (title,'w') as fileOutput:
        fileOutput.write(html + "\n")
    

In [36]:
#Question 3:
import feedparser
dict_list = []
fields = ["channel","url","title","date"]
count =0

for x in rss_list:
    d = feedparser.parse(rss_list[count])
    for key in d["entries"]:
        values = []
        values.append(rss_list[count])
        values.append(key["link"])
        values.append(key["title"])
        values.append(key["published"])
        dict_list.append(zip(fields,values))
    count += 1
    
print dict_list


[[('channel', 'http://www.volkskrant.nl/nieuws/rss.xml'), ('url', u'http://www.volkskrant.nl/economie/leegstand-vend-panden-drama-voor-winkelstraten-kleinere-steden~a4246375/'), ('title', u"Leegstand V&D-panden 'drama voor winkelstraten kleinere steden'"), ('date', u'Wed, 17 Feb 2016 09:12:00 GMT')], [('channel', 'http://www.volkskrant.nl/nieuws/rss.xml'), ('url', u'http://www.volkskrant.nl/buitenland/-is-vormt-geen-bedreiging-voor-ons-bestaan~a4245749/'), ('title', u"'IS vormt geen bedreiging voor ons bestaan'"), ('date', u'Wed, 17 Feb 2016 09:00:00 GMT')], [('channel', 'http://www.volkskrant.nl/nieuws/rss.xml'), ('url', u'http://www.volkskrant.nl/buitenland/berlijn-gaat-leerkrachten-werven-in-nederland~a4246396/'), ('title', u'Berlijn gaat leerkrachten werven in Nederland'), ('date', u'Wed, 17 Feb 2016 08:54:00 GMT')], [('channel', 'http://www.volkskrant.nl/nieuws/rss.xml'), ('url', u'http://www.volkskrant.nl/buitenland/-china-installeert-raketten-op-betwist-eiland-in-zuid-chinese-ze

In [38]:
#Question 4:
print "there are:",len(dict_list),"items in the rss feed"


there are: 1169 items in the rss feed


In [ ]:
#Question 5:


### JSON parsing

1. Download <http://maartenmarx.nl/teaching/ISatWork/NoteBooks/consuming-json-data-from-a-web-service.ipynb>, remove all code blocks, and turn it into a notebook again. 
2. Check that what you did is correct and you did not remove too much using a notebook viewer.
3. Now extract all code from the downloaded notebook, save it to a file, and execute it as a Python script. Does it give errors? Is it syntactically correct?

### PDF parsing

1. Save a wordfile as PDF, open it in Python, extract all text. Describe the differences, if any. Try the same with a two column PDF file from the web. This exercise gets more interesting if you use _difficult_ PDF. Why not try <http://wch.github.io/latexsheet/latexsheet.pdf>?

* Is the word order still as it should be?
* What about the strange characters?

# Part 2 Python recap

Download [PythonRecap2.0.ipynb](PythonRecap2.0.ipynb),  and answer all questions as asked.